In [8]:
# Notebook settings
from importlib import reload
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [9]:
# Default
import os
import sys
module_path = os.path.abspath('/mnt/cmnfs/proj/ORIGINS/protMSD/maxquant/ScanByScan/')
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd

In [10]:
# path to MaxQuant results (evidence.txt) of the same RAW file, used for constructing reference dictionary
maxquant_file_dict = '/mnt/cmnfs/proj/ORIGINS/data/ecoli/HpHRP/MQ/100FDR_PSM_protein/'\
    'combined/txt/evidence_1_FilteredByClosestRT_transfer_RT_pred_filtered_withIso.pkl' 
maxquant_file_exp = '/mnt/cmnfs/proj/ORIGINS/data/ecoli/ss/DDA/MQ/combined/txt/evidence_1.txt'
maxquant_file_dict_expRT = maxquant_file_dict[:-4]+"_expRT.pkl"
print(maxquant_file_dict_expRT)

/mnt/cmnfs/proj/ORIGINS/data/ecoli/HpHRP/MQ/100FDR_PSM_protein/combined/txt/evidence_1_FilteredByClosestRT_transfer_RT_pred_filtered_withIso_expRT.pkl


In [11]:
# Load maxquant result (reference)
Maxquant_result_dict = pd.read_pickle(filepath_or_buffer=maxquant_file_dict)
Maxquant_result_exp = pd.read_csv(filepath_or_buffer=maxquant_file_exp, sep='\t')

In [12]:
# prepare maxquant exp result
Maxquant_result_exp_for_merge = Maxquant_result_exp[['Modified sequence', 'Charge', 'Retention time']]
Maxquant_result_exp_for_merge = Maxquant_result_exp_for_merge.groupby(['Modified sequence', 'Charge']).\
    agg({'Retention time':'median'}).reset_index()

In [13]:
Maxquant_result_dict_exp_RT = pd.merge(left=Maxquant_result_dict,
                                       right=Maxquant_result_exp_for_merge,
                                       on = ['Modified sequence', 'Charge'],
                                       suffixes=[None, '_exp'],
                                       how='left')
Maxquant_result_dict_exp_RT['Retention time'] = Maxquant_result_dict_exp_RT['Retention time_exp']
Maxquant_result_dict_exp_RT['Retention time new'] = Maxquant_result_dict_exp_RT['Retention time_exp'].\
    fillna(Maxquant_result_dict_exp_RT['predicted_RT'])
Maxquant_result_dict_exp_RT.to_pickle(maxquant_file_dict_expRT)
